In [1]:
from quantdev.data import Databank
# from quantdev.trade import PortfolioManager, Position, Portfolio
import quantdev.backtest as bts

from typing import Literal
import pandas as pd
import numpy as np

In [8]:
roe = bts.get_factor('roe')
test = bts.backtesting(roe>0.99, 'QR', benchmark=['0050'], stop_loss=-0.7, stop_at='intraday')

In [9]:
test.report

BokehModel(combine_events=True, render_bundle={'docs_json': {'08c0d7f1-ed8c-4e57-a21d-5ee15d2054b0': {'version…

In [5]:
roe = bts.get_factor('roe')
test = bts.backtesting(roe>0.99, 'QR', benchmark=['0050'])
test.report

BokehModel(combine_events=True, render_bundle={'docs_json': {'17fd4904-39f1-45fb-976a-d46bd254eaa0': {'version…

In [12]:
return_df = test.return_df
portfolio_df = test.portfolio_df
buy_list = test.buy_list

DatetimeIndex(['2005-04-01', '2005-05-16', '2005-08-15', '2005-11-15',
               '2006-04-03', '2006-05-16', '2006-08-15', '2006-11-15',
               '2007-04-02', '2007-05-16', '2007-08-15', '2007-11-15',
               '2008-04-01', '2008-05-16', '2008-08-15', '2008-11-17',
               '2009-04-01', '2009-05-18', '2009-08-17', '2009-11-16',
               '2010-04-01', '2010-05-17', '2010-08-16', '2010-11-15',
               '2011-04-01', '2011-05-16', '2011-08-15', '2011-11-15',
               '2012-04-02', '2012-05-16', '2012-08-15', '2012-11-15',
               '2013-04-01', '2013-05-16', '2013-08-15', '2013-11-15',
               '2014-04-01', '2014-05-16', '2014-08-15', '2014-11-17',
               '2015-04-01', '2015-05-18', '2015-08-17', '2015-11-16',
               '2016-04-01', '2016-05-16', '2016-08-15', '2016-11-15',
               '2017-04-05', '2017-05-16', '2017-08-15', '2017-11-15',
               '2018-04-02', '2018-05-16', '2018-08-15', '2018-11-15',
      

In [48]:
def _stop_loss_or_profit(buy_list, portfolio_df, return_df, pct:float, stop_at:Literal['intraday', 'next_day']='intraday'):
    # calculate portfolio returns by multiplying portfolio positions (0/1) with return data
    portfolio_return = (portfolio_df != 0).astype(int) * return_df[portfolio_df.columns].loc[portfolio_df.index]
    
    # initialize empty DataFrame to store portfolio after stop loss/profit adjustments
    aftermath_portfolio_df = pd.DataFrame(index=portfolio_return.index, columns=portfolio_return.columns)
    
    # get rebalancing period dates
    period_starts = portfolio_df.index.intersection(buy_list.index)
    period_ends = period_starts[1:].map(lambda x: portfolio_df.loc[:x].index[-1])
    period_ends = period_ends.append(pd.Index([portfolio_return.index[-1]]))

    # convert DataFrames to numpy arrays for faster computation
    returns_array = portfolio_return.values
    dates_array = portfolio_return.index.values

    # process each rebalancing period
    for start_date, end_date in zip(period_starts, period_ends):
        # get array indices for current period
        start_idx = portfolio_return.index.get_indexer([start_date])[0]
        end_idx = portfolio_return.index.get_indexer([end_date])[0] + 1
        print(start_date, end_date)
        print(start_idx, end_idx)
        
        # extract returns and dates for current period
        period_returns = returns_array[start_idx:end_idx]
        period_dates = dates_array[start_idx:end_idx]
        
        # calculate cumulative returns for period
        cum_returns = np.cumprod(1 + period_returns, axis=0) - 1
        
        # create mask for stop loss/profit triggers based on threshold
        if pct<0:
            stop_loss_mask = cum_returns <= pct  # stop loss
        else:
            stop_loss_mask = cum_returns >= pct  # stop profit
        
        # skip if period has no data
        if len(stop_loss_mask) == 0:
            aftermath_portfolio_df.iloc[start_idx:end_idx] = portfolio_df.values[start_idx:end_idx]
            continue
        
        # find first trigger point for each stock
        trigger_idx = np.argmax(stop_loss_mask, axis=0)
        has_trigger = np.any(stop_loss_mask, axis=0)
        
        # for 'next_day' mode, shift trigger point one day forward
        if stop_at=='next_day':
            trigger_idx = np.where(has_trigger & (trigger_idx < len(period_dates) - 1), trigger_idx + 1, trigger_idx)
        
        # create mask for dates after trigger points
        date_matrix = period_dates[:, None] > period_dates[trigger_idx]
        
        # only apply mask to stocks that hit stop loss/profit
        final_mask = np.where(has_trigger, date_matrix, False)
        
        # get portfolio values and zero out positions after stop loss/profit
        period_portfolio = portfolio_df.values[start_idx:end_idx]
        filtered_portfolio = np.where(final_mask, 0, period_portfolio)
        
        # store results
        aftermath_portfolio_df.iloc[start_idx:end_idx] = filtered_portfolio

    return aftermath_portfolio_df

In [45]:
portfolio_df

,1101,1102,1103,1104,1107,1108,1109,1110,1201,1203,...,9944,9945,9946,9949,9950,9951,9955,9958,9960,9962
t_date,,,,,,,,,,,,,,,,,,,,,
2005-05-16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2005-05-17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2005-05-18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2005-05-19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2005-05-20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-02-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-02-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
result_stop_loss = _stop_loss_or_profit(
    buy_list=buy_list,
    portfolio_df=portfolio_df,
    return_df=return_df,
    pct=-0.99,
    stop_at='intraday'
)

2005-05-16 00:00:00 2005-08-15 00:00:00
0 64
2005-08-15 00:00:00 2005-11-15 00:00:00
63 128
2005-11-15 00:00:00 2006-04-03 00:00:00
127 220
2006-04-03 00:00:00 2006-05-16 00:00:00
219 249
2006-05-16 00:00:00 2006-08-15 00:00:00
248 313
2006-08-15 00:00:00 2006-11-15 00:00:00
312 377
2006-11-15 00:00:00 2007-04-02 00:00:00
376 467
2007-04-02 00:00:00 2007-05-16 00:00:00
466 497
2007-05-16 00:00:00 2007-08-15 00:00:00
496 561
2007-08-15 00:00:00 2007-11-15 00:00:00
560 624
2007-11-15 00:00:00 2008-04-01 00:00:00
623 714
2008-04-01 00:00:00 2008-05-16 00:00:00
713 745
2008-05-16 00:00:00 2008-08-15 00:00:00
744 809
2008-08-15 00:00:00 2008-11-17 00:00:00
808 873
2008-11-17 00:00:00 2009-04-01 00:00:00
872 963
2009-04-01 00:00:00 2009-05-18 00:00:00
962 995
2009-05-18 00:00:00 2009-08-17 00:00:00
994 1058
2009-08-17 00:00:00 2009-11-16 00:00:00
1057 1123
2009-11-16 00:00:00 2010-04-01 00:00:00
1122 1214
2010-04-01 00:00:00 2010-05-17 00:00:00
1213 1245
2010-05-17 00:00:00 2010-08-16 00:00:

In [33]:
start_date='2023-11-21'
end_date='2023-11-22'
portfolio_df.values #[start_date:end_date]


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [50]:
x = (result_stop_loss * return_df).sum(axis=1)
(x+1).cumprod() - 1

t_date
2005-01-03           0
2005-01-04           0
2005-01-05           0
2005-01-06           0
2005-01-07           0
                ...   
2025-02-03    9.291614
2025-02-04    9.417635
2025-02-05    9.461422
2025-02-06     9.46782
2025-02-07    9.471729
Length: 4943, dtype: object

In [22]:
result_stop_loss.infer_objects(copy=False).replace(np.nan, 0).apply(lambda row: np.count_nonzero(row), axis=1)

t_date
2005-05-16    0
2005-05-17    0
2005-05-18    0
2005-05-19    0
2005-05-20    0
             ..
2025-02-03    5
2025-02-04    5
2025-02-05    5
2025-02-06    5
2025-02-07    5
Length: 4857, dtype: int64

In [ ]:


spl_df = portfolio_df.map(lambda x: 1 if x>0 else 0) * return_df
# Calculate cumulative returns between rebalance dates with stop loss
stop_loss_returns = pd.DataFrame(index=spl_df.index, columns=spl_df.columns)

for i in range(len(test.buy_list.index)):
    start_date = test.buy_list.index[i]
    if start_date == test.buy_list.index[-1]:
        end_date = spl_df.index[-1]
    else:
        end_date = test.buy_list.index[i+1] - pd.Timedelta(days=1)
        
    period_returns = spl_df.loc[start_date:end_date]
    
    # Calculate cumulative returns for this period
    cum_returns = (1 + period_returns).cumprod() - 1
    
    # Find where stop loss is triggered (-10%)
    stop_loss_mask = cum_returns <= -0.1
    
    # For each stock that hits stop loss, get first trigger date and fill forward
    # For each stock column in the stop_loss_mask DataFrame
    for col in stop_loss_mask.columns:
        # Check if this stock ever hits the stop loss (-10%) in this period
        if stop_loss_mask[col].any():
            # Get the first date where stop loss is triggered for this stock
            # idxmax() returns the index of the first True value
            first_trigger = stop_loss_mask[col].idxmax()
            
            # Set all values after the trigger date to True
            # This marks all dates after stop loss is hit
            stop_loss_mask.loc[first_trigger:, col] = True
    
    # Apply stop loss by zeroing out returns after trigger
    period_returns[stop_loss_mask] = 0
    
    # Recalculate cumulative returns with stop loss applied
    stop_loss_returns.loc[start_date:end_date] = (1 + period_returns).cumprod() - 1


In [30]:
spl_df = portfolio_df.map(lambda x: 1 if x>0 else 0) * return_df
stop_loss_returns = pd.DataFrame(index=spl_df.index, columns=spl_df.columns)

# Get all period start and end dates
period_starts = test.buy_list.index
period_ends = period_starts[1:].append(pd.Index([spl_df.index[-1]]))

for start_date, end_date in zip(period_starts, period_ends):
    period_returns = spl_df.loc[start_date:end_date]
    
    # Calculate cumulative returns for this period
    cum_returns = (1 + period_returns).cumprod() - 1
    
    # Find first occurrence of stop loss trigger for each stock
    stop_loss_mask = cum_returns <= -0.1
    first_triggers = stop_loss_mask.apply(lambda x: x.idxmax() if x.any() else pd.NaT)

    date_range = period_returns.index
    stop_loss_mask = pd.DataFrame(
        [[d > trigger if pd.notna(trigger) else False 
          for trigger in first_triggers] 
         for d in date_range],
        index=date_range,
        columns=period_returns.columns
    ) 
    
    # Apply stop loss by zeroing out returns after trigger
    period_returns[stop_loss_mask] = 0
    
    # Calculate final cumulative returns with stop loss applied
    stop_loss_returns.loc[start_date:end_date] = (1 + period_returns).cumprod() - 1

In [70]:

# Create sample data
# Let's create 5 trading days and 3 stocks
dates = pd.date_range('2024-01-01', '2024-01-10')
stocks = ['2330', '2317', '2345']

# Create portfolio_df (1 means we hold the stock, 0 means we don't)
portfolio_data = {
    '2330': [0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.5, 0.5, 0.5, 0.5],
    '2317': [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5], 
    '2345': [0.5, 0.5, 0.5, 0.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0]
}
portfolio_df = pd.DataFrame(portfolio_data, index=dates)

# Create return_df (daily returns for each stock)
return_data = {
    '2330': [0.02, -0.04, -0.03, 0.01, 0.02, 0.01, -0.02, 0.03, -0.01, 0.02],
    '2317': [-0.03, -0.05, 0.02, -0.02, 0.01, -0.01, 0.02, -0.02, 0.03, 0.01],
    '2345': [0.01, 0.02, -0.08, 0.03, 0.02, 0.01, -0.03, 0.02, 0.01, -0.02]
}
return_df = pd.DataFrame(return_data, index=dates)

# Create buy_list (rebalancing dates DataFrame)
buy_data = {
    '2330': [0, 1],
    '2317': [1, 1],
    '2345': [1, 0]
}
buy_list = pd.DataFrame(buy_data, index=[dates[0], dates[5]])

# Test the function
# 1. Test stop loss (-10%)
result_stop_loss = _stop_loss_or_profit(
    buy_list=buy_list,
    portfolio_df=portfolio_df,
    return_df=return_df,
    pct=-0.05,
    stop_at='intraday'
)

# Print results
print("Original Portfolio:")
print(portfolio_df)
# print("\nDaily Returns:")
# print(return_df)
print("\nBuy List (Rebalancing Dates):")
print(buy_list)
print("\nCumulative Returns:")
print((1 + return_df).cumprod() - 1)
print("\nStop Loss Result (-10%):")
print(result_stop_loss)

# # Test with take profit (10%)
# result_take_profit = _stop_loss_or_profit(
#     buy_list=buy_list,
#     portfolio_df=portfolio_df,
#     return_df=return_df,
#     pct=0.05,
#     stop_at='intraday'
# )
# print("\nTake Profit Result (10%):")
# print(result_take_profit)

Original Portfolio:
            2330  2317  2345
2024-01-01   0.0   0.5   0.5
2024-01-02   0.0   0.5   0.5
2024-01-03   0.0   0.5   0.5
2024-01-04   0.0   0.5   0.5
2024-01-05   0.0   0.5   0.5
2024-01-06   0.5   0.5   0.0
2024-01-07   0.5   0.5   0.0
2024-01-08   0.5   0.5   0.0
2024-01-09   0.5   0.5   0.0
2024-01-10   0.5   0.5   0.0

Buy List (Rebalancing Dates):
            2330  2317  2345
2024-01-01     0     1     1
2024-01-06     1     1     0

Cumulative Returns:
                2330      2317      2345
2024-01-01  0.020000 -0.030000  0.010000
2024-01-02 -0.020800 -0.078500  0.030200
2024-01-03 -0.050176 -0.060070 -0.052216
2024-01-04 -0.040678 -0.078869 -0.023782
2024-01-05 -0.021491 -0.069657 -0.004258
2024-01-06 -0.011706 -0.078961  0.005699
2024-01-07 -0.031472 -0.060540 -0.024472
2024-01-08 -0.002416 -0.079329 -0.004961
2024-01-09 -0.012392 -0.051709  0.004989
2024-01-10  0.007360 -0.042226 -0.015111

Stop Loss Result (-10%):
           2330 2317 2345
2024-01-01  0.0  0.

In [57]:
result_take_profit

,2330,2317,2345
2024-01-01,0.0,0.5,0.5
2024-01-02,0.0,0.5,0.5
2024-01-03,0.0,0.5,0.5
2024-01-04,0.0,0.5,0.5
2024-01-05,0.0,0.5,0.5
2024-01-06,NaN,NaN,NaN
2024-01-07,NaN,NaN,NaN
2024-01-08,NaN,NaN,NaN
2024-01-09,NaN,NaN,NaN
2024-01-10,NaN,NaN,NaN


In [51]:
import pandas as pd
import numpy as np

# Create a sample DataFrame
data = {
    'A': [1, 3, 5, 7],
    'B': [4, 2, 6, 8],
    'C': [2, 2, 2, 2]
}
df = pd.DataFrame(data) >= 4

# Use np.argmax to find the index of the maximum value in each column
max_indices = np.argmax(df.values, axis=0)

# Check if each column has any True value
has_true = df.any(axis=0)

# Replace indices with -1 where there are no True values
max_indices = np.where(has_true, max_indices, -1)

print("DataFrame:")
print(df)
print("\nIndices of maximum values in each column (or -1 if no True values):")
print(max_indices)

DataFrame:
       A      B      C
0  False   True  False
1  False  False  False
2   True   True  False
3   True   True  False

Indices of maximum values in each column (or -1 if no True values):
[2 0 0]


In [73]:
import pandas as pd
import numpy as np

# Create a sample DataFrame
data = {
    'A': [1, 3, 5, 7],
    'B': [4, 2, 6, 8],
    'C': [2, 2, 2, 2]
}
df = pd.DataFrame(data)

# Use np.where to create a new DataFrame based on a condition
# For example, set values to 'High' if they are greater than 4, otherwise 'Low'
condition = df > 4
result = np.where(condition, 'High', 'Low')

# Convert the result back to a DataFrame
result_df = pd.DataFrame(result, columns=df.columns)

print("Original DataFrame:")
print(df)
print("\nCondition (df > 4):")
print(condition)
print("\nResult DataFrame with 'High' and 'Low':")
print(result_df)


Original DataFrame:
   A  B  C
0  1  4  2
1  3  2  2
2  5  6  2
3  7  8  2

Condition (df > 4):
       A      B      C
0  False  False  False
1  False  False  False
2   True   True  False
3   True   True  False

Result DataFrame with 'High' and 'Low':
      A     B    C
0   Low   Low  Low
1   Low   Low  Low
2  High  High  Low
3  High  High  Low
